## (6) IJV - GRID import to GDB from excel GRID template

In [3]:
import arcpy
import openpyxl
import os
import pandas as pd
import zipfile
from arcpy import metadata as md



### 1) Import excel file to GDB

In [23]:
excel_file_path =  r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\1_INPUT\GRID\20231020_EAC_layout 21MW_BC_from_Daniel_Kartar\Layout1_Libra III BC2 50x21MW 7 loops 3xs.xlsx"

output_gdb_path = r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\GRID\GRID.gdb"

output_fc = "NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1" 

In [25]:

def find_table_in_excel(file_path, keyword):
    # Load the workbook
    wb = openpyxl.load_workbook(file_path)

    # Loop through each sheet in the workbook
    for sheet in wb.sheetnames:
        ws = wb[sheet]

        # Loop through each row in the sheet
        for row in ws.iter_rows():
            # Loop through each cell in the row
            for cell in row:
                # Check if the cell value matches the keyword
                if cell.value == keyword:
                    start_cell = cell.coordinate

                    # Get start cell coordinates
                    start_col = start_cell[0]
                    start_row = int(start_cell[1:])

                    # Initialize end cell coordinates
                    end_col = start_col
                    end_row = start_row

                    # Find end of table in rows
                    while ws[end_col + str(end_row)].value is not None:
                        end_row += 1

                    # Find end of table in columns
                    while ws[end_col + str(start_row)].value is not None:
                        if ord(end_col) < 90:  # ASCII value of 'Z'
                            end_col = chr(ord(end_col) + 1)
                        else:
                            break

                    # Adjust for overstepping boundaries
                    if ord(end_col) > 65:  # ASCII value of 'A'
                        end_col = chr(ord(end_col) - 1)
                        
                    end_cell = end_col + str(end_row - 1)

                    print(f"Excel table detected in sheet \"{sheet}\" and range {start_cell}:{end_cell}")
                    arcpy.AddMessage(f"Excel table detected in sheet \"{sheet}\" and range {start_cell}:{end_cell}")

                    # Return table range
                    return sheet, start_cell, end_cell                  
                    
def excel_table_to_feature_class(excel_file_path, output_gdb_path, output_fc, sheet_name, start_cell, end_cell, epsg_code):
    
    epsg_code = int(epsg_code)
    
    # Load the workbook and select the sheet
    wb = openpyxl.load_workbook(excel_file_path)
    ws = wb[sheet_name]

    # Create an empty list to store the cable strings and a dictionary to store the attributes
    cable_strings = []
    attributes = {}
    string_number = 0

    # Get the headers from the first row of the cell range
    headers = [cell.value for cell in ws[start_cell:end_cell][0]]
    
    # As the coordinate columns for start and end point have same names, they need to get renamed to be unique
    replacements = {'Easting [m]': ['Start Easting [m]', 'End Easting [m]'], 
                    'Northing [m]': ['Start Northing [m]', 'End Northing [m]'],
                    'Depth to LAT [m]': ['Start Depth to LAT [m]', 'End Depth to LAT [m]']}
    for key, values in replacements.items():
        try:
            for value in values:
                index = headers.index(key)
                headers[index] = value
        except ValueError:
            pass        

    print(f'Header row from Excel: {headers}')
    arcpy.AddMessage(f'Header row from Excel: {headers}')

    # Create a dictionary to map valid field names to original headers
    field_names = {arcpy.ValidateFieldName(header): header for header in headers}
    field_names = {name.rstrip("_").replace("__","_"): value for name, value in field_names.items()}
    
    # Iterate through each row in the cell range (excluding the header row)
    for row in ws[start_cell:end_cell][1:]:
        # Create a polyline geometry object using the start and end coordinates
        array = arcpy.Array([arcpy.Point(row[1].value, row[2].value), arcpy.Point(row[5].value, row[6].value)])
        polyline = arcpy.Polyline(array)
    
        # Check if the start point is 0 and increment string number
        if row[0].value == 0:
            string_number += 1
    
        # Create a dictionary of attributes for the current row
        attrs = {'String_number' : string_number}
        attrs.update({header: value for header, value in zip(field_names, row)})
    
        # Append a tuple with the polyline and the attributes to the list
        cable_strings.append((polyline, attrs))

    # Define the spatial reference of the output feature class (you may need to adjust this)
    spatial_reference = arcpy.SpatialReference(epsg_code)

    print(f'Create feature class: {output_fc}')
    arcpy.AddMessage(f'Create feature class: {output_fc}')

    # Create a new feature class in the geodatabase
    fc_path = os.path.join(output_gdb_path, output_fc)
    arcpy.CreateFeatureclass_management(output_gdb_path, output_fc, 'POLYLINE', spatial_reference=spatial_reference)

    # String number should be the first attribute in fc table, so an extra step is needed to get it added to field_names dict
    string_item = ({"String_number" : "String number"})
    field_names = {**string_item, **field_names}
    print(field_names)

    # Add fields for each header in the Excel table
    # arcpy.AddField_management(fc_path, 'string_number', 'LONG', field_alias="String number")
    for field_name, header in field_names.items():
        if header in ['Start point', 'End point', "String number"]:
            arcpy.AddField_management(fc_path, field_name, 'SHORT', field_alias=header)
        else:
            arcpy.AddField_management(fc_path, field_name, 'DOUBLE', field_alias=header)

    # Use an insert cursor to add the cable strings to the new feature class
    print(f'Write cable strings to fc')
    arcpy.AddMessage(f'Write cable strings to fc')
    
    try:
        with arcpy.da.InsertCursor(fc_path, ['SHAPE@'] + list(field_names.keys())) as cursor:
            for cable_string, attrs in cable_strings:
                cursor.insertRow([cable_string] + [attrs[field_name].value if hasattr(attrs[field_name], 'value') else attrs[field_name] for field_name in field_names])
    except Exception as e:
        print(f"An error occurred: {e}")
        arcpy.AddError(f"An error occurred: {e}")
    print(f"*** Finished ***")
    arcpy.AddMessage(f"*** Finished ***")
   
arcpy.env.workspace = output_gdb_path
keyword = "Start point"

# Call function to get the sheet name and cell range
sheet_name, start_cell, end_cell = find_table_in_excel(excel_file_path, keyword)

# Call function to 
excel_table_to_feature_class(excel_file_path, output_gdb_path, output_fc, sheet_name, start_cell, end_cell, 25831) 


Excel table detected in sheet "Sheet1" and range B17:U74
Header row from Excel: ['Start point', 'Start Easting [m]', 'Start Northing [m]', 'Start Depth to LAT [m]', 'End point', 'End Easting [m]', 'End Northing [m]', 'End Depth to LAT [m]', 'A [m]', 'B [m]', 'C [m]', 'D [m]', 'E [m]', 'Length [m] incl. Water depth', 'Cross section [mm²]', 'Material', 'CAPEX [€]', 'OPEX [€]', 'Loss [MWh]', 'Loss [%]']
Create feature class: NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1
{'String_number': 'String number', 'Start_point': 'Start point', 'Start_Easting_m': 'Start Easting [m]', 'Start_Northing_m': 'Start Northing [m]', 'Start_Depth_to_LAT_m': 'Start Depth to LAT [m]', 'End_point': 'End point', 'End_Easting_m': 'End Easting [m]', 'End_Northing_m': 'End Northing [m]', 'End_Depth_to_LAT_m': 'End Depth to LAT [m]', 'A_m': 'A [m]', 'B_m': 'B [m]', 'C_m': 'C [m]', 'D_m': 'D [m]', 'E_m': 'E [m]', 'Length_m_incl_Water_depth': 'Length [m] incl. Water depth', 'Cross_section_mm': 'Cross section [mm²]'

### 2) optional add Type of cabel as a new field Backlink/ArrayCable

In [26]:
output_fc_gdb = os.path.join(output_gdb_path, output_fc)

# Name of the new field and its data type
new_field_name = "Type_cable"
field_type = "TEXT"  
arcpy.AddField_management(output_fc_gdb, new_field_name, field_type)


# Names of the fields you want to update
field_condition = "OPEX"

# Start an update cursor and update the field based on the condition
with arcpy.da.UpdateCursor(output_fc_gdb, [field_condition, new_field_name]) as cursor:
    for row in cursor:
        if row[0] == 0:  
            row[1] = "Backlink"
        else:  # If field contains a different number, set it to 2
            row[1] = "Array Cable"
        cursor.updateRow(row)
print("Field updated successfully!")

Field updated successfully!


### 3) Metadata from Excel Files

In [27]:
excel_file = r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\5_PROJECTDOCS\NL_IJV_Dataset_Inventory.xlsx"

# String to search for
df = pd.read_excel(excel_file)
search_string  = output_fc


df.columns

# Find the row where the search_string is in search_column
filtered_df = df[df["Full Name"] == output_fc]
filtered_df


c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,Full Name,Project Prefix,Name,Type,Version,Primary Category,Sub-Categories,Coordinate System,CoordSysAbbv,EPSG,Date Created,Comment,Summary,Description,Tags,Credits,Added to Portal
186,NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1,NL_IJV_III,EAC_L007_276D_DA6_50T_v0_loops,li,v1,GRID,Array Cables,ETRS 89 UTM31 N,UTM31N,25834,2023-10-20 00:00:00,NaN,NaN,v1 - GRID layout for business case 2 based on ...,"Grid, Cable",andrea.sulova@rwe.com,Y


In [28]:


# Check if the search_string was found in the DataFrame
if not filtered_df.empty:
    # Get the text from text_column in the same row
    imported_title = filtered_df.iloc[0]["Full Name"]
    imported_summary = filtered_df.iloc[0]["Summary"]
    imported_tags = filtered_df.iloc[0]["Tags"]
    imported_Description = filtered_df.iloc[0]["Description"]
    imported_Credits = filtered_df.iloc[0]["Credits"]
    imported_Date = str(filtered_df.iloc[0]["Date Created"])

    imported_Description = imported_Description + "\n" + imported_Date

    # Print or use the imported_text as needed
    print("imported_title:", imported_title)
    print("imported_summary:", imported_summary)
    print("imported_tags:", imported_tags)
    print("imported_Description:", imported_Description)
    print("imported_Credits:", imported_Credits)

else:
    print("Search string not found in the specified column.")

# Create a new Metadata object and add some content to it
new_md = md.Metadata()
new_md.title = imported_title
new_md.tags = imported_tags
new_md.summary = imported_summary
new_md.description = imported_Description
new_md.credits = imported_Credits

# Assign the Metadata object's content to a target item
tgt_item_md = md.Metadata(output_fc_gdb)
if not tgt_item_md.isReadOnly:
    tgt_item_md.copy(new_md)
    tgt_item_md.save()



imported_title: NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1
imported_summary: nan
imported_tags: Grid, Cable
imported_Description: v1 - GRID layout for business case 2 based on developable area DA6.   Both Alpha and Beta from are  loop layouts .Data sent by: daniel.kajtar@rwe.com \\WM20ocqu46ph01\WF_Projects\NL_IJV\1_INPUT\GRID\20231020_EAC_layout 21MW_BC_from_Daniel_Kartar
2023-10-20 00:00:00
imported_Credits: andrea.sulova@rwe.com


### 4) Your New Alias Name

In [29]:

# Get the current alias name (optional)
# Get the describe object for the feature class
desc = arcpy.Describe(output_fc_gdb)
current_alias = desc.aliasName
print(current_alias)


new_alias = output_fc.replace("_", " ")
print(new_alias)

try:
    # Set the new alias name for the feature class
    arcpy.AlterAliasName(output_fc_gdb, new_alias)
    print(f"Alias name for {output_fc_gdb} changed from '{current_alias}' to '{new_alias}' successfully.")

except Exception as e:
    print(f"An error occurred: {str(e)}")



NL IJV III EAC L005 276D DA6 50T loops UTM31N v1
Alias name for \\WM20ocqu46ph01\WF_Projects\NL_IJV\2_FINAL\GRID\GRID.gdb\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1 changed from '' to 'NL IJV III EAC L005 276D DA6 50T loops UTM31N v1' successfully.


### 5) Export to Shapefiles

In [30]:
shp_path = r"\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_WTG_GRID_layouts"

shp_folder = os.path.join(shp_path, output_fc)
shp_file = output_fc + ".shp"
output_shp = os.path.join(shp_folder,shp_file )


# Check if the folder already exists before creating it
if not os.path.exists(shp_folder):
    os.makedirs(shp_folder)
    arcpy.FeatureClassToFeatureClass_conversion(output_fc_gdb, shp_folder, shp_file)
    print(f"Folder '{shp_folder}' created successfully.")
else:
    print(f"Folder '{shp_file}' already exists.")


Folder '\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_WTG_GRID_layouts\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1' created successfully.


#### 6) Zip folder

In [31]:
zip_folder_path = shp_folder + ".zip"

def zip_folder(shp_folder, zip_folder_path):
    with zipfile.ZipFile(zip_folder_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, dirs, files in os.walk(shp_folder):
            for file in files:
                zip_file.write(os.path.join(root, file), file)

# Example usage

if not os.path.exists(zip_folder_path):
    zip_folder(shp_folder, zip_folder_path)
    print('Final output Zip:', zip_folder_path)
else:
    print(f"Zip folder '{zip_folder_path}' already exists.")


Final output Zip: \\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_WTG_GRID_layouts\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1.zip


#### 7) Export file to DWG file for CAD


In [32]:
dwg_folder = os.path.join(shp_folder+ '_DWG')
print("DWG Folder: ", dwg_folder)

shp_file_path = os.path.join(shp_folder, shp_file)
print("SHP File:", shp_file_path)

# Output DWG file
dwg_output = os.path.join(dwg_folder, output_fc +'.dwg')
print("DWG File:", dwg_output)

if not os.path.exists(dwg_folder):
    os.makedirs(dwg_folder)
    arcpy.conversion.ExportCAD(shp_file_path, "DWG_R2018", dwg_output, False, False)
    print(f"Folder '{dwg_folder}' created successfully.")
else:
    print(f"Folder '{dwg_folder}' already exists.")


DWG Folder:  \\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_WTG_GRID_layouts\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1_DWG
SHP File: \\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_WTG_GRID_layouts\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1.shp
DWG File: \\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_WTG_GRID_layouts\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1_DWG\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1.dwg
Folder '\\WM20ocqu46ph01\WF_Projects\NL_IJV\4_OUTPUT\_WTG_GRID_layouts\NL_IJV_III_EAC_L005_276D_DA6_50T_loops_UTM31N_v1_DWG' created successfully.
